# Import

In [243]:
import pandas as pd

In [244]:
import matplotlib.pyplot as plt

# 한글 폰트 지정
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

font_path = "C:/Windows/Fonts/malgun.ttf"

# Load Data

In [245]:
train_df = pd.read_csv("../../data/train_df_translate_del.csv")
test_df = pd.read_csv("../../data/test_df_translate_del.csv")  
submission = pd.read_csv("../../data/sample_submission.csv")  

## 전처리

영어로만 구성된 단어에 대한 처리 (주소 제거)

In [246]:
import pandas as pd

# 제거할 단어 리스트
remove_words = ['http', 'www.', '.kr', '.net', '.com']

# NaN 값을 빈 문자열로 대체
train_df['키워드'] = train_df['키워드'].fillna('')
test_df['키워드'] = test_df['키워드'].fillna('')

# 필터링 조건 생성
filter_condition_train = train_df['키워드'].apply(lambda x: not any(word in x for word in remove_words))
filter_condition_test = test_df['키워드'].apply(lambda x: not any(word in x for word in remove_words))

# 필터링된 데이터프레임 생성 및 원본 데이터프레임에 반영
train_df = train_df[filter_condition_train]
test_df = test_df[filter_condition_test]

In [247]:
import pandas as pd
import re

# 영어로만 된 단어를 필터링하는 함수
def is_english(word):
    return bool(re.match('^[a-zA-Z]+$', word))

# 영어로만 된 단어를 추출하는 함수
def extract_english_keywords(keywords):
    words = keywords.split(',')
    english_words = [word for word in words if is_english(word)]
    return english_words

# 모든 영어 단어를 하나의 리스트로 모으기
all_english_words = []
train_df['키워드'].apply(lambda x: all_english_words.extend(extract_english_keywords(x)))

# 리스트를 시리즈로 변환하여 value_counts 호출
english_word_counts = pd.Series(all_english_words).value_counts()

# 상위 10개의 결과 출력
print(english_word_counts.head(10))

SK       1405
SNS      1155
LH       1015
AI        910
KLPGA     897
GTX       746
YTN       744
TF        738
NH        720
CCTV      711
dtype: int64


In [248]:
# Pandas 출력 옵션 설정
pd.set_option('display.max_rows', None)

english_word_counts.head(20)

SK                                       1405
SNS                                      1155
LH                                       1015
AI                                        910
KLPGA                                     897
GTX                                       746
YTN                                       744
TF                                        738
NH                                        720
CCTV                                      711
IC                                        710
TV                                        541
GH                                        442
KG                                        368
CC                                        309
ICT                                       301
CJ                                        294
MOU                                       292
LED                                       282
KB                                        254
MBC                                       236
ESG                               

In [249]:
len(english_word_counts)

2207

갯수가 2개이하인 단어 제거

In [251]:
import pandas as pd
import re

# 영어로만 된 단어를 필터링하는 함수
def is_english(word):
    return bool(re.match('^[a-zA-Z]+$', word))

# 영어로만 된 단어를 추출하는 함수
def extract_english_keywords(keywords):
    words = keywords.split(',')
    english_words = [word for word in words if is_english(word)]
    return english_words

# 모든 영어 단어를 하나의 리스트로 모으기
train_english_words = []
test_english_words = []

train_df['키워드'].apply(lambda x: train_english_words.extend(extract_english_keywords(x)))
test_df['키워드'].apply(lambda x: test_english_words.extend(extract_english_keywords(x)))

# 리스트를 시리즈로 변환하여 value_counts 호출
train_english_word_counts = pd.Series(train_english_words).value_counts()
test_english_word_counts = pd.Series(test_english_words).value_counts()

# 2개 이하로 등장하는 단어 제거
train_words_to_remove = train_english_word_counts[train_english_word_counts <= 2].index.tolist()
test_words_to_remove = test_english_word_counts[test_english_word_counts <= 2].index.tolist()

# 원본 데이터에서 해당 단어들을 제거하는 함수
def remove_infrequent_words(keywords, words_to_remove):
    words = keywords.split(',')
    filtered_words = [word for word in words if word not in words_to_remove]
    return ','.join(filtered_words)

# 원본 데이터에 반영
train_df['키워드'] = train_df['키워드'].apply(lambda x: remove_infrequent_words(x, train_words_to_remove))
test_df['키워드'] = test_df['키워드'].apply(lambda x: remove_infrequent_words(x, test_words_to_remove))

# 상위 10개의 결과 출력
filtered_train_english_word_counts = train_english_word_counts[train_english_word_counts > 2]
filtered_test_english_word_counts = test_english_word_counts[test_english_word_counts > 2]

print("Train DataFrame 상위 10개의 결과:")
print(filtered_train_english_word_counts.head(10))

print("\nTest DataFrame 상위 10개의 결과:")
print(filtered_test_english_word_counts.head(10))

Train DataFrame 상위 10개의 결과:
SK       1405
SNS      1155
LH       1015
AI        910
KLPGA     897
GTX       746
YTN       744
TF        738
NH        720
CCTV      711
dtype: int64

Test DataFrame 상위 10개의 결과:
SK       588
SNS      465
LH       464
AI       451
KLPGA    361
GTX      352
TF       309
YTN      298
IC       297
CCTV     280
dtype: int64


In [261]:
filtered_train_english_word_counts.head(20)

SK       1405
SNS      1155
LH       1015
AI        910
KLPGA     897
GTX       746
YTN       744
TF        738
NH        720
CCTV      711
IC        710
TV        541
GH        442
KG        368
CC        309
ICT       301
CJ        294
MOU       292
LED       282
KB        254
dtype: int64

In [252]:
len(filtered_train_english_word_counts)

882

In [253]:
len(filtered_test_english_word_counts)

514

In [257]:
# 정확히 '경기용' 단어를 포함하고 있는 행 필터링 / 경기용인플랫폼 이런 단어는 찾지 않음
filtered_df = train_df[train_df['키워드'].str.contains(r'\bCity\b', na=False)]

# 결과 출력
len(filtered_df)

22

.